# OpenAPI agents

We can construct agents to consume arbitrary APIs, here APIs conformant to the OpenAPI/Swagger specification.

## 1st example: "json explorer" agent

Here's an agent that's not particularly practical, but neat! The agent has access to 2 toolkits. One comprises tools to interact with json: one tool to list the keys of a json object and another tool to get the value for a given key. The other toolkit comprises `requests` wrappers to send GET and POST requests. This agent consumes a lot calls to the language model, but does a surprisingly decent job.

Here's an example interacting with OpenAI's OpenAPI spec 😝.

In [1]:
import os
import yaml
from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import RequestsWrapper
from langchain.tools.json.tool import JsonSpec

In [2]:
with open("openai_openapi.yml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec=JsonSpec(dict_=data, max_value_length=4000)
headers = {
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"
}
requests_wrapper=RequestsWrapper(headers=headers)
openapi_toolkit = OpenAPIToolkit.from_llm(OpenAI(temperature=0), json_spec, requests_wrapper, verbose=True)
openapi_agent_executor = create_openapi_agent(
    llm=OpenAI(temperature=0),
    toolkit=openapi_toolkit,
    verbose=True
)

In [3]:
openapi_agent_executor.run("Make a post request to openai /completions. The prompt should be 'tell me a joke.'")



> Entering new AgentExecutor chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new AgentExecutor chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the servers key to see what the base url is
Action: json_spec_list_keys
Action Input: data["servers"][0]
Observation: ValueError('Value at path `data["servers"][0]` is not a dict, get the value directly.')
Thought: I should get the value of the servers key
Action: json_spec_get_value
Action Input: data["servers"][0]
Observation: {'url': 'https://api.openai.com/v1'}
Thought: I now know the base url for the API
Final Answer: The base url for the API is https://api.openai.com/v1

> Finished chain.

Observation: The base url for the API is https://api.openai.com/v1
Thought: I should find the path for the /completions endpoint.
Action: json_explorer
Action Input: What is the path for th

'The response of the POST request to the /completions endpoint with the prompt \'tell me a joke\' is {"id":"cmpl-6zzlQk2P9UaB6U0N0Qn8Fvy4DHkKw","object":"text_completion","created":1680233448,"model":"davinci","choices":[{"text":"?\\" more to fill the air than anything else and was pleasantly surprised ... maybe the","index":0,"logprobs":null,"finish_reason":"length"}],"usage":{"prompt_tokens":4,"completion_tokens":16,"total_tokens":20}}.'

## 2nd example: hierarchical planning agent

- In this example, we'll consider a more principle approach called hierarchical planning, common in robotics and appearing in recent works for LLMs X robotics.
- We'll see it allows us to work with a massive API spec AND to assist with user queries that require multiple steps against the API.
- To get better results + to save on tokens, we'll separate concerns: a "planner" will be responsible for what endpoints to call and a "controller" will be responsible for how to call them.
- In the initial implementation, the planner is an LLM chain that has the name and a short description for each endpoint in context. The controller is an LLM agent that is instantiated with documentation for only the endpoints for a particular plan.

---

We'll work with the Spotify API. There's a bit of auth-related setup to do if you want to replicate this.

- You'll have to set up an application in the Spotify developer console, documented [here](https://developer.spotify.com/documentation/general/guides/authorization/), to get credentials: `CLIENT_ID`, `CLIENT_SECRET`, and `REDIRECT_URI`.
- To get an access tokens (and keep them fresh), you can implement the oauth flows, or you can use `spotipy`. If you've set your Spotify creedentials as environment variables `SPOTIPY_CLIENT_ID`, `SPOTIPY_CLIENT_SECRET`, and `SPOTIPY_REDIRECT_URI`, you can use the helper functions below:

In [4]:
import os, yaml, subprocess
import spotipy.util as util

SPOTIFY_API_SPEC_URL = "https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml"

def fetch_spotify_api_spec():    
    if not os.path.exists('./openapi.yaml'):
        subprocess.run(["wget", SPOTIFY_API_SPEC_URL])
    with open("./openapi.yaml", "r") as f:
        spec = yaml.load(f, Loader=yaml.Loader)
    return spec

def construct_spotify_auth_headers(raw_spec: dict):
    scopes = list(raw_spec['components']['securitySchemes']['oauth_2_0']['flows']['authorizationCode']['scopes'].keys())
    access_token = util.prompt_for_user_token(scope=','.join(scopes))
    return {
        'Authorization': f'Bearer {access_token}'
    }

In [5]:
# Get the API spec.
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
raw_spotify_api_spec = fetch_spotify_api_spec()
spotify_api_spec = reduce_openapi_spec(raw_spotify_api_spec, dereference=False)

In [6]:
# Get API credentials.
from langchain.requests import RequestsWrapper
headers = construct_spotify_auth_headers(raw_spotify_api_spec)
requests_wrapper = RequestsWrapper(headers=headers)

#### How big is this spec?

In [7]:
endpoints = [
    (route, operation)
    for route, operations in raw_spotify_api_spec["paths"].items()
    for operation in operations
    if operation in ["get", "post"]
]
len(endpoints)

63

In [8]:
import tiktoken
enc = tiktoken.encoding_for_model('text-davinci-003')
def count_tokens(s): return len(enc.encode(s))

count_tokens(yaml.dump(raw_spotify_api_spec))

80326

#### Let's see some examples!

Starting with GPT-4. (Some robustness iterations under way for GPT-3 family.)

In [10]:
from langchain.llms.openai import OpenAI
from langchain.agents.agent_toolkits.openapi import planner

llm = OpenAI(model_name="gpt-4", temperature=0.0)
agent = planner.create_openapi_agent(spotify_api_spec, requests_wrapper, llm)
user_query = "make me a playlist with the first song from kind of blue. call it machine blues."
agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to create a playlist with the first song from Kind of Blue and name it Machine Blues
Observation: 1. GET /search to find the album ID for Kind of Blue.
2. GET /albums/{id}/tracks to get the tracks from the album and retrieve the ID of the first song.
3. GET /me to get the current user's ID.
4. POST /users/{user_id}/playlists to create a new playlist named "Machine Blues" for the current user.
5. POST /playlists/{playlist_id}/tracks to add the first song from Kind of Blue to the newly created playlist.
Thought:I'm ready to execute the API calls.
Action: api_controller
Action Input: 1. GET /search to find the album ID for Kind of Blue.
2. GET /albums/{id}/tracks to get the tracks from the album and retrieve the ID of the first song.
3. GET /me to get the current user's ID.
4. POST /users/{user_id}/playlists to create a new playlist named "Machine Blues" for the current user.
5. PO

'I have created a playlist called "Machine Blues" with the first song from Kind of Blue. Enjoy!'

In [11]:
user_query = "give me a song I'd like, make it blues-ey"
agent.run(user_query)



> Entering new AgentExecutor chain...
Action: api_planner
Action Input: I need to find the right API calls to get a blues song recommendation for the user
Observation: 1. GET /recommendations/available-genre-seeds to find valid recommendation seeds and check if "blues" is available.
2. GET /recommendations with the "blues" genre seed to find a blues song recommendation.
Thought:I have the plan, now I need to execute the API calls.
Action: api_controller
Action Input: 1. GET /recommendations/available-genre-seeds to find valid recommendation seeds and check if "blues" is available.
2. GET /recommendations with the "blues" genre seed to find a blues song recommendation.

> Entering new AgentExecutor chain...
Action: requests_get
Action Input: {"url": "https://api.spotify.com/v1/recommendations/available-genre-seeds", "output_instructions": "extract the list of available genres"}
Observation: acoustic, afrobeat, alt-rock, alternative, ambient, anime, black-metal, bluegrass, blues, bossa

'The recommended blues song for you is "Wang Dang Doodle" with the ID: 1LEacXCo9v6TtcAiOpehWc.'